In [1]:
import torch 
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn import  ensemble, preprocessing, metrics
from sklearn.base import  clone
import matplotlib
import seaborn
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from torch.utils.data import Dataset,DataLoader
from xgboost import XGBClassifier
from torch import nn
import os
BASEPATH="../input/agriculture-master-competition"

In [2]:
def seed_torch(seed=31):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_torch()

In [3]:
def read_csvfile(filepath):
    df = pd.read_csv(filepath)
    return df
input_df = read_csvfile(os.path.join(BASEPATH,"train_data.csv"))
input_df_tmp = read_csvfile(os.path.join(BASEPATH,"train_data.csv"))
labelcolumns = list(input_df.columns[20:])
labelcolumns.remove('actuator09')
featurecolumns = list(input_df.columns[1:20])
featurecolumns.remove("d.rainfall_detect")
#featurecolumns = ["d.log_time","d.temperature_B","d.humidity_A","d.soil_temperature","d.soil_humidity","d.soil_EC","d.soil_PH","d.outside_temperature","d.outside_humidity","d.radiometric","d.rainfall","d.wind_speed"]
#featurecolumns.remove("d.rainfall")
#featurecolumns.remove("d.wind_direction")
print(featurecolumns)
random_state = 31

['d.log_time', 'd.temperature_A', 'd.temperature_B', 'd.humidity_A', 'd.humidity_B', 'd.photometric', 'd.CO2', 'd.soil_temperature', 'd.soil_humidity', 'd.soil_EC', 'd.soil_PH', 'd.outside_photometric', 'd.outside_temperature', 'd.outside_humidity', 'd.wind_speed', 'd.wind_direction', 'd.radiometric', 'd.rainfall']


In [4]:
def GetLogDate(input_df):
    input_df["d.log_date"] = [int(i.split(' ')[0].split('/')[1].zfill(2)+i.split(' ')[0].split('/')[2].zfill(2))  for i in input_df["d.log_time"].tolist()]
    return input_df
def GetLogTime(input_df):
    input_df["d.log_time"] = [int(i.split(' ')[1].replace(':','')) for i in input_df["d.log_time"].tolist()]
    return input_df
def AvgSameData(input_df):
    input_df["d.temperature"] = (input_df["d.temperature_A"]+input_df["d.temperature_B"])/2
    input_df["d.humidity"] = (input_df["d.humidity_A"]+input_df["d.humidity_B"])/2
    return input_df
print(input_df["d.log_time"][0])
input_df=GetLogDate(input_df)
input_df=GetLogTime(input_df)
print(input_df["d.log_date"][0])
print(input_df["d.log_time"][0])
featurecolumns.append("d.log_date")

#input_df = AvgSameData(input_df)
#featurecolumns.remove("d.temperature_A")
#featurecolumns.remove("d.temperature_B")
#featurecolumns.remove("d.humidity_A")
#featurecolumns.remove("d.humidity_B")
#featurecolumns.append("d.temperature")
#featurecolumns.append("d.humidity")
input_df.describe()

2020/7/31 00:05
731
5


,index,d.log_time,d.temperature_A,d.temperature_B,d.humidity_A,d.humidity_B,d.photometric,d.CO2,d.soil_temperature,d.soil_humidity,...,actuator03,actuator04,actuator05,actuator06,actuator07,actuator08,actuator09,actuator10,actuator11,d.log_date
count,28493.000000,28493.000000,28493.000000,28493.000000,28493.000000,28493.000000,28493.000000,28493.000000,28493.000000,28493.000000,...,28493.000000,28493.000000,28493.000000,28493.000000,28493.000000,28493.000000,28493.000000,28493.000000,28493.000000,28493.000000
mean,14246.000000,1184.833819,28.002109,26.713484,69.650862,80.268452,6411.977889,454.088899,24.755951,47.510834,...,0.989647,0.981399,0.873232,0.873688,0.873829,0.645106,0.668726,0.038395,0.024989,1007.647317
std,8225.364946,692.977576,6.398441,5.569665,14.808369,12.576656,9743.290448,35.413986,4.422882,4.636636,...,0.101225,0.135114,0.332719,0.332206,0.332048,0.478490,0.470680,0.192152,0.156093,147.038716
min,0.000000,0.000000,10.000000,9.300000,22.800000,43.900000,0.000000,376.000000,7.300000,36.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,731.000000
25%,7123.000000,600.000000,23.400000,22.700000,57.100000,69.900000,0.000000,428.000000,21.700000,44.800000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,902.000000
50%,14246.000000,1205.000000,27.500000,26.800000,72.900000,82.700000,23.000000,449.000000,25.000000,47.500000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1010.000000
75%,21369.000000,1805.000000,32.300000,30.400000,82.000000,90.800000,11310.000000,474.000000,27.700000,50.500000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1123.000000
max,28492.000000,2355.000000,50.600000,41.400000,93.800000,99.900000,65530.000000,602.000000,35.200000,62.600000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1231.000000


In [5]:
def logData(df,columnsname):
    df[columnsname]=df[columnsname].apply(lambda x: x if x!=0 else 000.1)
    df[columnsname] = np.log(df[columnsname]).fillna(0)
    return df
#input_df = logData(input_df,"d.wind_direction")
#input_df[featurecolumns].describe()

In [6]:
xgbf = XGBClassifier(n_estimators=30)
#rffeature = RandomForestRegressor(n_estimators = 50, random_state = 31)
def replacezerovalues(df,columnsname,train=True):
    featureX = df[df[columnsname] > 0][featurecolumns].copy()
    LabelX =featureX[columnsname].copy()
    featureX = featureX.drop(columns=columnsname)
    
    
    if train:
        #rffeature.fit(featureX,LabelX)
        xgbf.fit(np.array(featureX), np.array(LabelX))
    
    featureY = df[featurecolumns].copy()
    featureY = featureY.drop(columns=columnsname)
    #print(featureY.columns,len(featureY.columns))
    
    
    newLabelY = xgbf.predict(np.array(featureY))
    #newLabelY = rffeature.predict(featureY)
    df[columnsname] = newLabelY
    return df

columnsnames=["d.wind_speed"]
for idx,columnsname in enumerate(columnsnames):
    input_df = replacezerovalues(input_df,columnsname)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:53:54] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [7]:
def dataNormalized(feature_df,zeromean=True):
    if zeromean:
        feature_df = (feature_df - feature_df.mean())/feature_df.std()
    else:
        feature_df=(feature_df-feature_df.min())/(feature_df.max()-feature_df.min())
    return feature_df
#input_df[featurecolumns] = dataNormalized(input_df[featurecolumns],False)

In [8]:
def balanceDataset(train_df,valid_df,minnum):
    #for lc in labelcolumns:
    #    print(len(train_df[train_df[lc] == 0]),len(valid_df[valid_df[lc] == 0]),len(train_df[train_df[lc] == 1]),len(valid_df[valid_df[lc] == 1]))
    #print("===")
    for lc in labelcolumns:
        l=0 if len(valid_df[valid_df[lc] == 1]) > len(valid_df[valid_df[lc] == 0]) else 1
        trainn = len(train_df[train_df[lc] == l])
        validn=len(valid_df[valid_df[lc] == l])
        if validn+1 < minnum*(trainn+validn):
            moven = (trainn+validn)*minnum-validn
            move_rows = train_df[train_df[lc] == l].sample(frac=moven/trainn).copy()
            valid_df=valid_df.append(move_rows)
            train_df=train_df.drop(move_rows["index"])

            validnn=len(valid_df[valid_df[lc] == (not l)])
            if validnn > len(move_rows):
                move_rows = valid_df[valid_df[lc] == (not l)].sample(frac=len(move_rows)/validnn).copy()
                train_df=train_df.append(move_rows)
                valid_df=valid_df.drop(move_rows["index"])
    #for lc in labelcolumns:
    #    print(len(train_df[train_df[lc] == 0]),len(valid_df[valid_df[lc] == 0]),len(train_df[train_df[lc] == 1]),len(valid_df[valid_df[lc] == 1]))
    return train_df,valid_df


def splitDataframe(df,train_sample):
    shuffle_df = df.sample(frac=1,random_state=31)
    train_df,valid_df = np.split(shuffle_df,[int(train_sample*len(shuffle_df))])
    #print(len(train_df),len(valid_df))
    train_df,valid_df=balanceDataset(train_df,valid_df,1-train_sample)
    #print(len(train_df),len(valid_df))
    return train_df,valid_df
            
train_df,valid_df=splitDataframe(input_df,0.975)
print(len(train_df),len(valid_df))
#featureX = dataNormalized(train_df[featurecolumns],True)
#featureY = dataNormalized(valid_df[featurecolumns],True)

27780 713


In [9]:
def trainDataframeBalance(df):
    balancecolumns = labelcolumns[:6]
    balance_df = pd.DataFrame(columns = df.columns)
    for cn in balancecolumns:
        balance_df=balance_df.append(df[df[cn] == 0].copy())
        
    balancecolumns = labelcolumns[-2:]
    for cn in balancecolumns:
        balance_df=balance_df.append(df[df[cn] == 1].copy())
        
    #showactuatorplot(balance_df)
    balancecolumns = labelcolumns[:6]
    for cn in balancecolumns:
        tmp_zero=balance_df[balance_df[cn] == 0]
        tmp_one=balance_df[balance_df[cn] == 1]
        if 2*len(tmp_one)<len(tmp_zero):
            balance_df=balance_df.append(df[df[cn] == 1].sample(frac=(len(tmp_zero)-len(tmp_one))/(len(tmp_zero)*3)))
    #showactuatorplot(balance_df)
    return balance_df
#balance_df=trainDataframeBalance(train_df)
        

In [10]:
def removeOutlier(input_df,times=3):
    input_df["d.wind_speed"] = input_df["d.wind_speed"].apply(lambda x: 0 if x<0 else x)
    input_df = input_df[abs(input_df["d.photometric"]-input_df["d.photometric"].mean())<input_df["d.photometric"].std()*times]
    input_df = input_df[abs(input_df["d.outside_photometric"]-input_df["d.outside_photometric"].mean())<input_df["d.outside_photometric"].std()*times]
    input_df = input_df[abs(input_df["d.radiometric"]-input_df["d.radiometric"].mean())<input_df["d.radiometric"].std()*times]
    
    return input_df
train_df = removeOutlier(train_df,5)



In [11]:
def splitDataset(df):
    featureX = df[featurecolumns]
    labelX = df[labelcolumns]
    return featureX,labelX
featureX,labelX = splitDataset(train_df)
featureY,labelY = splitDataset(valid_df)
labelX09 = train_df["actuator09"]
labelY09 = valid_df["actuator09"]

In [12]:
def printperformance(predictions,label):
    accuracy = metrics.accuracy_score(label, predictions)
    print(accuracy)
    print(metrics.precision_recall_fscore_support(label, predictions,average='micro'))

## Firstmodel


In [13]:
#model = RandomForestClassifier(n_estimators = 100, random_state = 31)
#from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve,KFold


modelExtra = ExtraTreesClassifier(criterion='entropy',n_estimators=130,min_impurity_decrease=0.0,min_weight_fraction_leaf=0.0,min_samples_leaf=1,min_samples_split=3,n_jobs=-1,random_state=31,max_depth=None,max_features= 'auto')
modelRandomforest = RandomForestClassifier(criterion='entropy',n_estimators=70,min_impurity_decrease=0.0,min_weight_fraction_leaf=0.0,min_samples_leaf=1,min_samples_split=3,n_jobs=-1,random_state=31,max_depth=None,max_features= 'sqrt')
#cv_results=cross_val_score(model, featureX, y = labelX, scoring = "f1_score", cv = 10, n_jobs=-1)

modelExtra.fit(featureX,labelX)
predictions = modelExtra.predict_proba(featureY)
predictions = np.array(predictions)[:,:,1]
predictionsExtra=np.moveaxis(predictions, -1, 0)
#for idx,p in enumerate(predictionsExtra):
#    predictionsExtra[idx] = [ 1 if i>=0.5 else 0 for i in p]
#printperformance(labelY,predictionsExtra)

modelRandomforest.fit(featureX,labelX)
predictions = modelRandomforest.predict_proba(featureY)
predictions = np.array(predictions)[:,:,1]
predictionsRandomforest=np.moveaxis(predictions, -1, 0)
#for idx,p in enumerate(predictionsRandomforest):
#    predictionsRandomforest[idx] = [ 1 if i>=0.5 else 0 for i in p]
#printperformance(labelY,predictionsRandomforest)

predictionsmix = (predictionsRandomforest+predictionsExtra)/2

for idx,p in enumerate(predictionsmix):
    predictionsmix[idx] = [ 1 if i>=0.5 else 0 for i in p]
printperformance(labelY,predictionsmix)

#gb_param_grid = {
#              'min_impurity_decrease' : [0.0,0.2,0.4,0.6,0.8],
#              }

#grid = GridSearchCV(model,param_grid = gb_param_grid,cv=2, scoring="accuracy", n_jobs= -1, verbose = 1)

#grid.fit(featureX,labelX)

#model = grid.best_estimator_

#print( grid.best_params_, grid.best_score_)


#predictions = model.predict(featureY)
#printperformance(labelY,predictions)


0.9368863955119214
(0.9914263445050663, 0.9980384464495881, 0.9947214076246335, None)


0.935072531586336
(0.991887454527895, 0.9973133941865735, 0.9945930241941993, None)

0.9365933551708002
(0.9920645140520877, 0.9974107099509654, 0.9947304287478111, None)

In [14]:
def modelCV(featureX,labelX,kflod=5):
    modelsName = ["SVC","DecisionTree","AdaBoost","Bagging","RandomForest","ExtraTrees","GradientBoosting","MultipleLayerPerceptron","XGBClassifier","LogisticRegression","KNeighbors","LinearDiscriminantAnalysis"]

    kfold = StratifiedKFold(n_splits=kflod)
    random_state = 31
    classifiers = []
    classifiers.append(SVC(random_state=random_state))
    classifiers.append(DecisionTreeClassifier(random_state=random_state))
    classifiers.append(AdaBoostClassifier(ExtraTreesClassifier(random_state=random_state),random_state=random_state))
    classifiers.append(BaggingClassifier(random_state=random_state))
    classifiers.append(RandomForestClassifier(random_state=random_state))
    classifiers.append(ExtraTreesClassifier(random_state=random_state))
    classifiers.append(GradientBoostingClassifier(random_state=random_state))
    classifiers.append(MLPClassifier(random_state=random_state))
    classifiers.append(XGBClassifier(random_state=random_state))
    classifiers.append(LogisticRegression(random_state = random_state))
    classifiers.append(KNeighborsClassifier())
    classifiers.append(LinearDiscriminantAnalysis())

    cv_results = []
    for classifier in classifiers :
        cv_results.append(cross_val_score(classifier, np.array(featureX), y = np.array(labelX), scoring = "accuracy", cv = kfold, n_jobs=4))

    cv_means = []
    cv_std = []
    for cv_result in cv_results:
        cv_means.append(cv_result.mean())
        cv_std.append(cv_result.std())
        print(modelsName[len(cv_means)-1])
        print(cv_means[len(cv_means)-1])

    cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":modelsName})

    g = sns.barplot("CrossValMeans","Algorithm",data = cv_res, palette="Set3",orient = "h",**{'xerr':cv_std})
    g.set_xlabel("Mean Accuracy")
    g = g.set_title("Cross validation scores")


In [15]:
class Stacking_Model(object):
    def __init__(self,classifiers,meta_model,random_state):
        self.classifiers = list(classifiers.values())
        self.meta_model=meta_model
        self.random_state=random_state
        self.modelsName = list(classifiers.keys())
        
    def train(self,trainX,trainY,testX,testY,n_splits=5,mutiple=False):
        
        self.save_model=[[] for i in range(len(self.modelsName))]
        self.n_splits = n_splits
        kfold = KFold(n_splits=n_splits,shuffle=False)
        
        meta_train_feature= np.zeros((trainX.shape[0], len(self.modelsName)))
        
        for model_idx in range(len(self.modelsName)):
            print(self.modelsName[model_idx])
            k=0
            for train_index , test_index in kfold.split(trainX):
                k+=1
                classifier = clone(self.classifiers[model_idx])

                train_feature = featureX.iloc[train_index]
                test_feature = featureX.iloc[test_index]
                train_label = trainY.iloc[train_index]
                test_label = trainY.iloc[test_index]

                classifier.fit(train_feature,train_label)
                self.save_model[model_idx].append(classifier)

                test_predict = classifier.predict_proba(test_feature)
                test_predict = [i[1]for i in test_predict]
                meta_train_feature[test_index,model_idx] = test_predict


                predicts=classifier.predict(testX)
                accuracy = metrics.accuracy_score(predicts, testY)
                print(f"KFlod-{k}:",accuracy)
                #print(metrics.precision_recall_fscore_support(labelY09, predicts,average='micro'))
        
        self.meta_train_feature = meta_train_feature
        self.train_meta(trainY,testX,testY)
        
    def train_meta(self,trainY,testX,testY):
        
        for k,model in self.meta_model.items():
            self.meta_model[k].fit(self.meta_train_feature,trainY)
            predictions_stacking=self.predict_sigle(k,testX)
            accuracy = metrics.accuracy_score(testY, predictions_stacking)
            print(f"Stack {k}:")
            print(accuracy)
            print(metrics.precision_recall_fscore_support(testY, predictions_stacking,average='micro'))
            
        estimators=[]
        for k,v in self.meta_model.items():
            estimators.append ((k,v))
            
        self.finalvotingmodel = VotingClassifier(estimators=estimators, voting='soft', n_jobs=-1)
        self.finalvotingmodel.fit(self.meta_train_feature,trainY)
        
        predictions_stacking=self.predict(testX)
        accuracy = metrics.accuracy_score(testY, predictions_stacking)
        print("Stacked final voted model")
        print(accuracy)
        print(metrics.precision_recall_fscore_support(testY, predictions_stacking,average='micro'))
        
    def getMetaFeature(self,features):
        meta_feature=[]
        for model_idx in range(len(self.modelsName)):
            
            predicts=[]
            
            for flododx in range(0,self.n_splits):
                predicts_tmp=self.save_model[model_idx][flododx].predict_proba(features)
                predicts_tmp = [i[1]for i in predicts_tmp]
                #predicts_tmp=self.save_model[model_idx][flododx].predict(features)
                predicts.append(predicts_tmp)
                
            predict = np.array(predicts).mean(axis=0)
            predict = [[i] for i in predict]
            
            if meta_feature==[]:
                meta_feature= predict
            else:
                meta_feature = np.concatenate((meta_feature,predict),axis=1)
                
        return meta_feature
    def predict_sigle(self,modelname,features):
        meta_feature = self.getMetaFeature(features)
        predictions_stacking = self.meta_model[modelname].predict(meta_feature)
        return predictions_stacking
    def predict(self,features):
        meta_feature = self.getMetaFeature(features)
        predictions_stacking = self.finalvotingmodel.predict(meta_feature)
        return predictions_stacking

In [16]:
classifiers = {"AdaBoost":AdaBoostClassifier(n_estimators=650,random_state=random_state),
               #"Bagging":BaggingClassifier(n_estimators=600, random_state=random_state),
               "RandomForest":RandomForestClassifier(n_estimators=450,random_state=random_state,n_jobs=-1),
               "ExtraTrees":ExtraTreesClassifier(criterion='gini',n_estimators=400,n_jobs=-1,random_state=random_state),
               "GradientBoosting":GradientBoostingClassifier(n_estimators=350, random_state=random_state),
               "XGBClassifier":XGBClassifier(n_estimators=30,random_state=random_state),
               "LinearDiscriminantAnalysis":LinearDiscriminantAnalysis(),
               "KNeighbors":KNeighborsClassifier(),
               "LogisticRegression":LogisticRegression(random_state = random_state),
               "DecisionTree":DecisionTreeClassifier(random_state=random_state),
               "SVC":SVC(random_state=random_state,probability=True),}
               #"MLP":MLPClassifier(random_state=random_state)}
meta_classifiers = {
               #"RandomForest":RandomForestClassifier(n_estimators=100,random_state=random_state,n_jobs=-1),
               #"ExtraTrees":ExtraTreesClassifier(criterion='gini',n_estimators=110,n_jobs=-1,random_state=random_state),
               "GradientBoosting":GradientBoostingClassifier(n_estimators=110, random_state=random_state),}
               #"XGBClassifier":XGBClassifier(n_estimators=30,random_state=random_state),
               #"SVC":SVC(random_state=random_state,probability=True),}
meta_model=GradientBoostingClassifier(n_estimators=110, random_state=random_state)
Stackingmodels=Stacking_Model(classifiers,meta_classifiers,random_state)
Stackingmodels.train(featureX,labelX09,featureY,labelY09,n_splits=15)
predictions_stack01 = Stackingmodels.predict(featureY)

AdaBoost
KFlod-1: 0.8274894810659187
KFlod-2: 0.8260869565217391
KFlod-3: 0.8345021037868162
KFlod-4: 0.8330995792426368
KFlod-5: 0.8288920056100981
KFlod-6: 0.8345021037868162
KFlod-7: 0.8302945301542777
KFlod-8: 0.8260869565217391
KFlod-9: 0.8316970546984572
KFlod-10: 0.8302945301542777
KFlod-11: 0.8330995792426368
KFlod-12: 0.8373071528751753
KFlod-13: 0.8302945301542777
KFlod-14: 0.8345021037868162
KFlod-15: 0.8288920056100981
RandomForest
KFlod-1: 0.8415147265077139
KFlod-2: 0.8401122019635343
KFlod-3: 0.847124824684432
KFlod-4: 0.844319775596073
KFlod-5: 0.8415147265077139
KFlod-6: 0.8429172510518934
KFlod-7: 0.844319775596073
KFlod-8: 0.8330995792426368
KFlod-9: 0.8415147265077139
KFlod-10: 0.8359046283309958
KFlod-11: 0.8359046283309958
KFlod-12: 0.8387096774193549
KFlod-13: 0.8499298737727911
KFlod-14: 0.8429172510518934
KFlod-15: 0.8415147265077139
ExtraTrees
KFlod-1: 0.820476858345021
KFlod-2: 0.8246844319775596
KFlod-3: 0.8330995792426368
KFlod-4: 0.82328190743338
KFlod-5: 

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


KFlod-1: 0.844319775596073
[17:12:04] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
KFlod-2: 0.847124824684432
[17:12:05] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
KFlod-3: 0.8485273492286115
[17:12:05] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
KFlod-4: 0.844319775596073
[17:12:06] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:lo

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


KFlod-1: 0.7643758765778401


...

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Stack GradientBoosting:
0.8541374474053296
(0.8541374474053296, 0.8541374474053296, 0.8541374474053295, None)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82:

Stacked final voted model
0.8541374474053296
(0.8541374474053296, 0.8541374474053296, 0.8541374474053295, None)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82:

In [17]:
class Vote_Model(object):
    def __init__(self,classifiers,random_state):
        self.classifiers = classifiers
        self.random_state=random_state
    def train(self,trainX,trainY,testX,testY):
        for k,classifier in self.classifiers.items():
            classifier.fit(np.array(trainX),np.array(trainY))
            predictions = classifier.predict(np.array(testX))
            accuracy = metrics.accuracy_score(testY, predictions)
            print(k)
            print(accuracy)
            print(metrics.precision_recall_fscore_support(testY, predictions,average='micro'))

        estimators=[]
        for k,v in self.classifiers.items():
            estimators.append ((k,v))
            
        self.votingmodel = VotingClassifier(estimators=estimators, voting='soft', n_jobs=-1)
        self.votingmodel.fit(trainX,trainY)
        
        predictions = self.predict(testX)
        accuracy = metrics.accuracy_score(testY, predictions)
        print("Voteed model")
        print(accuracy)
        print(metrics.precision_recall_fscore_support(testY, predictions,average='micro'))
    def predict(self,features):
        predictions = self.votingmodel.predict(np.array(features))
        return predictions


In [18]:
random_state=31
votemodels = {"AdaBoost":AdaBoostClassifier(n_estimators=350,random_state=random_state),
              "Bagging":BaggingClassifier(n_estimators=350, random_state=random_state),
              "RandomForest":RandomForestClassifier(n_estimators=350,random_state=random_state,n_jobs=-1),
              "ExtraTrees":ExtraTreesClassifier(criterion='entropy',n_estimators=350,n_jobs=-1,max_depth=None,random_state=random_state),
              "GradientBoosting":GradientBoostingClassifier(n_estimators=350, random_state=random_state),
              "XGBClassifier":XGBClassifier(n_estimators=30,random_state=random_state),
              "LinearDiscriminantAnalysis":LinearDiscriminantAnalysis()}
votemodel = Vote_Model(votemodels,random_state)
votemodel.train(featureX,labelX09,featureY,labelY09)

AdaBoost
0.8288920056100981
(0.8288920056100981, 0.8288920056100981, 0.8288920056100981, None)
Bagging
0.8401122019635343
(0.8401122019635343, 0.8401122019635343, 0.8401122019635343, None)
RandomForest
0.8373071528751753
(0.8373071528751753, 0.8373071528751753, 0.8373071528751753, None)
ExtraTrees
0.8302945301542777
(0.8302945301542777, 0.8302945301542777, 0.8302945301542777, None)
GradientBoosting
0.847124824684432
(0.847124824684432, 0.847124824684432, 0.847124824684432, None)
[18:16:05] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier
0.85273492286115
(0.85273492286115, 0.85273492286115, 0.8527349228611499, None)
LinearDiscriminantAnalysis
0.791023842917251
(0.791023842917251, 0.791023842917251, 0.791023842917251, None)
Voteed model
0.85273492286115
(0.85273492286115, 0.85273492286115, 0.8527349228611499, None)


In [19]:
predictions_vote = votemodel.predict(featureY)
accuracy = metrics.accuracy_score(labelY09, predictions_vote)
print("vote model")
print(accuracy)
print(metrics.precision_recall_fscore_support(labelY09, predictions_vote,average='micro'))

print("stack model")
predictions_stack = Stackingmodels.predict(featureY)
accuracy = metrics.accuracy_score(labelY09, predictions_stack)
print(accuracy)
print(metrics.precision_recall_fscore_support(labelY09, predictions_stack,average='micro'))



vote model
0.85273492286115
(0.85273492286115, 0.85273492286115, 0.8527349228611499, None)
stack model


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


0.8541374474053296
(0.8541374474053296, 0.8541374474053296, 0.8541374474053295, None)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


In [20]:
predictions09 = np.array([[i] for i in predictions_stack])
predictions=np.concatenate ((predictionsmix,predictions09),axis=1)
labelY["actuator09"] = labelY09

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
def calcaulateMacroF1(allpred,allans,allpredacc,nclasses,rou=3):
    recalls = [0 if allans[i] == 0 else 100*allpredacc[i]/allans[i] for  i in range(0,nclasses)]
    precisions = [0 if allpred[i] == 0 else 100*allpredacc[i]/allpred[i] for  i in range(0,nclasses)]
    avg_recalls = float(sum(recalls) / nclasses)
    avg_precisions = float(sum(precisions) / nclasses)
    beta=0.000001
    macro_f1 =(2+beta)*(avg_recalls*avg_precisions)/((avg_recalls+avg_precisions)+beta)
    macro_f1 = round(macro_f1,rou)
    precisions = [round(p,rou) for p in precisions]
    recalls = [round(r,rou) for r in recalls]
    return macro_f1,recalls ,precisions

In [22]:
if "actuator09" not in labelcolumns:
    labelcolumns.append("actuator09")
totalacc=[0]*len(labelcolumns)
totalans=[0]*len(labelcolumns)
totalpred=[0]*len(labelcolumns)
acc=0
#predictions = [[p] for p in predictions]
for idx in range(len(predictions)):
    allacc=True
    pred = [int(p>0.5) for p in predictions[idx]]
    if list(labelY.iloc[idx]) == list(pred):
        acc+=1
    for lidx in range(len(labelcolumns)):
        if labelcolumns[lidx] in input_df.columns[-2:] and False:
            totalans[lidx]+=not int(labelY.iloc[idx][lidx])
            predvalue = pred[lidx]
            totalpred[lidx]+= not predvalue
            if labelY.iloc[idx][lidx] == predvalue:
                totalacc[lidx]+= not predvalue
        else:
            totalans[lidx]+=int(labelY.iloc[idx][lidx])
            predvalue = pred[lidx]
            totalpred[lidx]+= predvalue
            if labelY.iloc[idx][lidx] == predvalue:
                totalacc[lidx]+= predvalue
                
print(acc/len(labelY))
macro_f1,recalls ,precisions = calcaulateMacroF1(totalpred,totalans,totalacc,len(labelcolumns))
print('f1-score: {} acc:{}'.format("%.3f"%macro_f1,"%.2f"%(acc/len(labelY))))
print('\nrecall: {} \nprecis: {}'.format(recalls,precisions))
print(sum(precisions)/len(precisions),sum(recalls)/len(recalls))
print(metrics.accuracy_score(labelY, predictions))
print(metrics.precision_recall_fscore_support(labelY, predictions,average='micro'))

0.8064516129032258
f1-score: 88.192 acc:0.81

recall: [100.0, 100.0, 100.0, 100.0, 99.836, 99.836, 99.836, 100.0, 3.704, 16.667, 94.017] 
precis: [99.856, 100.0, 100.0, 100.0, 99.672, 99.672, 99.672, 99.564, 50.0, 100.0, 85.271]
93.97336363636364 83.08145454545455
0.8064516129032258
(0.9846811542572141, 0.9871428571428571, 0.9859104690565365, None)


In [23]:
test_df = read_csvfile(os.path.join(BASEPATH,"test_data.csv"))
test_df=test_df.drop("d.rainfall_detect",axis=1)
print(featurecolumns)
test_df.info()

['d.log_time', 'd.temperature_A', 'd.temperature_B', 'd.humidity_A', 'd.humidity_B', 'd.photometric', 'd.CO2', 'd.soil_temperature', 'd.soil_humidity', 'd.soil_EC', 'd.soil_PH', 'd.outside_photometric', 'd.outside_temperature', 'd.outside_humidity', 'd.wind_speed', 'd.wind_direction', 'd.radiometric', 'd.rainfall', 'd.log_date']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12458 entries, 0 to 12457
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             12458 non-null  int64  
 1   d.log_time             12458 non-null  object 
 2   d.temperature_A        12458 non-null  float64
 3   d.temperature_B        12458 non-null  float64
 4   d.humidity_A           12458 non-null  float64
 5   d.humidity_B           12458 non-null  float64
 6   d.photometric          12458 non-null  float64
 7   d.CO2                  12458 non-null  float64
 8   d.soil_temperature     12458 non-nu

In [24]:
print(test_df["d.log_time"][0])
submit = read_csvfile(os.path.join(BASEPATH,"submission.csv"))
test_df["d.log_date"] = [int(d.split(' ')[0].split('-')[1]+d.split(' ')[0].split('-')[2]) for d in test_df["d.log_time"]]
print(test_df["d.log_date"][0])
test_df["d.log_time"] = [int(d.split(' ')[1].split(':')[0]+d.split(' ')[1].split(':')[1]) for d in test_df["d.log_time"]]
print(test_df["d.log_time"][0])
#test_df = logData(test_df,"d.wind_direction")
#test_df = AvgSameData(input_df)

for idx,columnsname in enumerate(columnsnames):
    test_df = replacezerovalues(test_df,columnsname,False)
    #test_df = logData(test_df,columnsname)

#normalized_feature = dataNormalized(test_df,True)

#test_df = normalized_feature


display(test_df.describe())
display(train_df[featurecolumns].describe())




2020-08-04 00:00:00
804
0


,Unnamed: 0,d.log_time,d.temperature_A,d.temperature_B,d.humidity_A,d.humidity_B,d.photometric,d.CO2,d.soil_temperature,d.soil_humidity,d.soil_EC,d.soil_PH,d.outside_photometric,d.outside_temperature,d.outside_humidity,d.wind_speed,d.wind_direction,d.radiometric,d.rainfall,d.log_date
count,12458.000000,12458.000000,12458.000000,12458.000000,12458.000000,12458.000000,12458.000000,12458.000000,12458.000000,12458.000000,12458.000000,12458.000000,12458.000000,12458.000000,12458.000000,12458.000000,12458.000000,12458.000000,12458.000000,12458.000000
mean,6228.500000,1171.971424,27.984315,26.604206,69.303131,80.309552,6164.260234,451.780703,24.760307,48.175341,0.623412,7.413646,8785.084765,26.411527,68.531907,1.418125,161.426794,195.066142,0.004880,1006.609889
std,3596.459161,693.796320,6.026032,5.132626,14.688278,11.909414,8983.605982,31.469165,3.917290,4.940224,0.174340,0.769050,15286.156576,5.379461,12.787051,1.037262,112.731940,261.863701,0.080669,137.232860
min,0.000000,0.000000,10.700000,10.600000,20.200000,36.700000,0.000000,383.000000,9.600000,37.100000,0.180000,6.300000,0.000000,10.600000,23.400000,0.700000,0.000000,0.000000,0.000000,804.000000
25%,3114.250000,550.000000,23.000000,22.300000,57.525000,70.700000,0.000000,428.000000,21.500000,44.300000,0.548000,6.600000,0.000000,22.200000,59.200000,0.700000,41.000000,0.000000,0.000000,905.000000
50%,6228.500000,1150.000000,27.300000,26.600000,72.700000,82.100000,23.000000,451.000000,25.200000,48.400000,0.663000,7.600000,33.500000,26.100000,68.800000,0.900000,168.000000,0.000000,0.000000,1011.000000
75%,9342.750000,1755.000000,32.200000,30.100000,81.400000,90.300000,11199.000000,471.000000,27.600000,50.700000,0.732000,8.000000,11731.750000,29.800000,77.700000,1.500000,261.000000,346.000000,0.000000,1115.000000
max,12457.000000,2355.000000,46.100000,40.200000,93.300000,99.600000,65339.000000,572.000000,33.400000,61.300000,1.014000,9.200000,65444.000000,42.000000,97.600000,8.700000,359.000000,949.000000,4.600000,1230.000000


,d.log_time,d.temperature_A,d.temperature_B,d.humidity_A,d.humidity_B,d.photometric,d.CO2,d.soil_temperature,d.soil_humidity,d.soil_EC,d.soil_PH,d.outside_photometric,d.outside_temperature,d.outside_humidity,d.wind_speed,d.wind_direction,d.radiometric,d.rainfall,d.log_date
count,27711.000000,27711.000000,27711.000000,27711.000000,27711.000000,27711.000000,27711.000000,27711.000000,27711.000000,27711.000000,27711.000000,27711.000000,27711.000000,27711.000000,27711.000000,27711.000000,27711.000000,27711.000000,27711.000000
mean,1183.510880,27.967735,26.690264,69.713439,80.320490,6288.329328,454.188986,24.740450,47.530887,0.607313,7.369752,8778.163220,26.496626,68.377998,1.637945,166.158168,197.905525,0.004302,1008.002707
std,693.311695,6.342871,5.537415,14.743313,12.524371,9391.160724,35.379117,4.406659,4.630938,0.147574,0.808801,15216.677021,5.787476,12.661607,1.379151,115.442587,265.163066,0.113022,146.818867
min,0.000000,10.000000,9.300000,22.800000,43.900000,0.000000,376.000000,7.300000,36.600000,0.131000,6.300000,0.000000,8.700000,22.300000,0.700000,0.000000,0.000000,0.000000,731.000000
25%,600.000000,23.400000,22.700000,57.200000,70.000000,0.000000,429.000000,21.700000,44.800000,0.520000,6.600000,0.000000,22.400000,59.100000,0.700000,42.000000,0.000000,0.000000,903.000000
50%,1205.000000,27.500000,26.800000,73.000000,82.700000,23.000000,449.000000,25.000000,47.500000,0.644000,7.600000,34.000000,26.200000,69.100000,1.100000,174.000000,0.000000,0.000000,1010.000000
75%,1800.000000,32.200000,30.300000,82.100000,90.800000,11226.000000,474.000000,27.700000,50.500000,0.708000,8.000000,11936.500000,29.900000,77.500000,1.900000,273.000000,352.000000,0.000000,1123.000000
max,2355.000000,50.600000,41.400000,93.800000,99.900000,55164.000000,602.000000,35.200000,62.600000,1.209000,9.200000,65500.000000,42.500000,98.000000,10.800000,359.000000,990.000000,9.200000,1231.000000


In [25]:

predictions = modelExtra.predict_proba(test_df[featurecolumns])
predictions = np.array(predictions)[:,:,1]
predictionsExtra=np.moveaxis(predictions, -1, 0)


predictions = modelRandomforest.predict_proba(test_df[featurecolumns])
predictions = np.array(predictions)[:,:,1]
predictionsRandomforest=np.moveaxis(predictions, -1, 0)


predictionsmix = (predictionsRandomforest+predictionsExtra)/2

for idx,p in enumerate(predictionsmix):
    predictionsmix[idx] = [ 1 if i>=0.5 else 0 for i in p]

predictions_stack = Stackingmodels.predict(test_df[featurecolumns])

predictions09 = np.array([[i] for i in predictions_stack])
predictions=np.concatenate ((predictionsmix,predictions09),axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:82:

In [26]:
ans_df = pd.DataFrame(predictions,columns=labelcolumns)
ans_df.to_csv("submission.csv")